In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.python.platform import gfile
import os
import time
import cv2
import pickle
sess=tf.InteractiveSession()
#face_img=np.load('face_10k.npy')

In [ ]:
/******************************************************************************
1.读取图片
2.提取脸部图片
3.将脸部图片输入到Inception--V3模型中提取特征
4.保存特征
/******************************************************************************

In [ ]:
def drew_face_eye(img):
        '''
        检测用户当前环境是否能够比较好的识别面部特征
        辨识出脸部以及眼部，标记出来
        :param img:
        :return:
        '''
        face_cascade = cv2.CascadeClassifier(r'D:\Proj_DL\Code\Proj_EyeTraker\haarcascade_frontalface_default.xml')
        eye_cascade = cv2.CascadeClassifier(r'D:\Proj_DL\Code\Proj_EyeTraker\haarcascade_eye.xml')
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3,5)
        #确保画面中只有一个人脸识别出来
        if(len(faces)!=1):
            return None
     
        for (x,y,w,h) in faces:
            
            roi_color = img[y:y+h, x:x+w]
#             #保存脸部图像
            face_mat=roi_color

            eye_mat=[]

        return  (img,face_mat,eye_mat)
file_root=r'D:\Proj_DL\Code\Proj_EyeTraker\Proj_iTraker\MIT_iTraker\calimg_file_num'
img_names=os.listdir(file_root)
#打乱顺序
index=np.arange(len(img_names))
np.random.shuffle(index)
img_names=list(np.array(img_names)[index])

In [ ]:
#输出
BOTTLE_NECK='pool_3/_reshape:0'
#jpg 二进制输入
JEPG_INPUT='DecodeJpeg/contents:0'
#矩阵输入
MAT_INPUT='ExpandDims:0'
#最后一个池化层的输入
POOL_3='mixed_10/join:0'
MODEL_DIR='inceptionv3_model/tensorflow_inception_graph.pb'

In [ ]:
with gfile.FastGFile(MODEL_DIR,'rb') as f:
    graph_def=tf.GraphDef()
    graph_def.ParseFromString(f.read())
feature_bottleneck,jpeg_tensor=tf.import_graph_def(graph_def,return_elements=[POOL_3,MAT_INPUT])

In [ ]:
#提取图片特征
y_cordinate=[]
face_matrix=[]
fail_counter=0
s_time=time.time()
file_index=0
for i,f in enumerate(img_names):
    lable_id=int(f[:-4].split('_')[-1])
    img=cv2.imread(os.path.join(file_root,f))
    face_op=drew_face_eye(img)
    if face_op == None:
        fail_counter+=1
        continue
    _,f,_=face_op
    f=f.reshape((1,f.shape[0],f.shape[1],f.shape[2]))
    fea_=sess.run(feature_bottleneck,{jpeg_tensor:f})
    lable_=[0]*16
    lable_[lable_id]=1
    y_cordinate.append(lable_)
    face_matrix.append(fea_.astype('float32'))
    if i%100==0:
        print('pic:{},fail count:{},run time:{}'.format(i,fail_counter,time.time()-s_time))
       # s_time=time.time()
    if (i%5000==0)and (i !=0):
        file_index+=1
        file_name='feature50k_pool3_'+str(file_index)+r'_.pkl'
        fp=open(os.path.join(r'Feature_File/',file_name),'wb')
        face_matrix=np.array(face_matrix).astype('float32')
        y_cordinate=np.array(y_cordinate).astype('uint8')
        pickle.dump(obj={'feature':face_matrix,'label':y_cordinate},file=fp)
        fp.close()
        y_cordinate=[]
        face_matrix=[]
        print('file save')
        #break

#y_cordinate=np.array(y_cordinate).astype('uint8')

In [ ]:
face_matrix=np.array(face_matrix).astype('float32').reshape((-1,2048))

In [ ]:
y_cordinate=np.array(y_cordinate).astype('uint8')

In [ ]:
import pickle

In [ ]:
fp=open('inception_10k.pkl','wb')
d={'feature':face_matrix,'label':y_cordinate}
pickle.dump(file=fp,obj=d)
fp.close()

In [ ]:
/*******************************************************************************************
训练MLP进行分类
/*******************************************************************************************

In [ ]:
#输入 X: (N,2048)
def model_MLP(x_input,traing):
    drop_rate=0.1
    with tf.variable_scope('layer_1'):
        w=tf.get_variable('weight',shape=[2048,1024],initializer=tf.contrib.layers.xavier_initializer())
        b=tf.get_variable('biases',initializer=tf.constant(0.1,shape=[1024,]))
        y_l1=tf.matmul(x_input,w)+b
        y_l1=tf.layers.batch_normalization(y_l1,training=traing)
        y_l1=tf.nn.relu(y_l1)
    with tf.variable_scope('layer_2'):
        y_l1=tf.nn.dropout(y_l1,drop_rate)
        w=tf.get_variable('weight',shape=[1024,512],initializer=tf.contrib.layers.xavier_initializer())
        b=tf.get_variable('biases',initializer=tf.constant(0.1,shape=[512,]))
        y_l2=tf.matmul(y_l1,w)+b
        y_l2=tf.layers.batch_normalization(y_l2,training=traing)
        y_l2=tf.nn.relu(y_l2)
    with tf.variable_scope('layer_3'):
        y_l2=tf.nn.dropout(y_l2,drop_rate)
        w=tf.get_variable('weight',shape=[512,16],initializer=tf.contrib.layers.xavier_initializer())
        b=tf.get_variable('biases',initializer=tf.constant(0.1,shape=[16,]))
        y_l3=tf.matmul(y_l2,w)+b
    return y_l3

In [ ]:
#输入 X: (N,8,8,2048)
def model_cnn(x_input,traing):
    drop_rate=0.3
    #conv1 3*3/1 (8,8,2048)->(6,6,2048)
    with tf.variable_scope('conv_1'):
        w=tf.get_variable('weight',shape=[3,3,2048,2048],initializer=tf.contrib.layers.xavier_initializer())
        b=tf.get_variable('biases',initializer=tf.constant(0.1,shape=[2048,]))
        conv_=tf.nn.conv2d(x_input,w,[1,1,1,1],'VALID')+b
        conv_=tf.layers.batch_normalization(conv_,training=traing)
        conv_=tf.nn.relu(conv_)
    #conv2 3*3/1 (6,6,2048)->(4,4,2048)
    with tf.variable_scope('conv_2'):
        w=tf.get_variable('weight',shape=[3,3,2048,2048],initializer=tf.contrib.layers.xavier_initializer())
        b=tf.get_variable('biases',initializer=tf.constant(0.1,shape=[2048,]))
        conv_=tf.nn.conv2d(conv_,w,[1,1,1,1],'VALID')+b
        conv_=tf.layers.batch_normalization(conv_,training=traing)
        conv_=tf.nn.relu(conv_)
     #conv3 3*3/1 (4,4,2048)->(2,2,2048)
    with tf.variable_scope('conv_3'):
        w=tf.get_variable('weight',shape=[3,3,2048,2048],initializer=tf.contrib.layers.xavier_initializer())
        b=tf.get_variable('biases',initializer=tf.constant(0.1,shape=[2048,]))
        conv_=tf.nn.conv2d(conv_,w,[1,1,1,1],'VALID')+b
        conv_=tf.layers.batch_normalization(conv_,training=traing)
        conv_=tf.nn.relu(conv_)
    #fc4 (N,8192)->(N,2048)
    with tf.variable_scope('fc_4'):
        x_flatten=tf.reshape(conv_,shape=[-1,2*2*2048]) # N*256
        w=tf.get_variable('weight',shape=[2*2*2048,2048],initializer=tf.contrib.layers.xavier_initializer())
        b=tf.get_variable('biases',initializer=tf.constant(0.1,shape=[2048,]))
        y_l1=tf.matmul(x_flatten,w)+b
        y_l1=tf.layers.batch_normalization(y_l1,training=traing)
        y_l1=tf.nn.relu(y_l1)
    #fc5 (N,2048)->(N,1024)
    with tf.variable_scope('fc_5'):
        y_l1=tf.nn.dropout(y_l1,drop_rate)
        w=tf.get_variable('weight',shape=[2048,1024],initializer=tf.contrib.layers.xavier_initializer())
        b=tf.get_variable('biases',initializer=tf.constant(0.1,shape=[1024,]))
        y_l2=tf.matmul(y_l1,w)+b
        y_l2=tf.layers.batch_normalization(y_l2,training=traing)
        y_l2=tf.nn.relu(y_l2)
    #fc6 (N,1024)->(N,16)
    with tf.variable_scope('fc_6'):
        y_l2=tf.nn.dropout(y_l2,drop_rate)
        w=tf.get_variable('weight',shape=[1024,16],initializer=tf.contrib.layers.xavier_initializer())
        b=tf.get_variable('biases',initializer=tf.constant(0.1,shape=[16,]))
        y_l3=tf.matmul(y_l2,w)+b
    return y_l3
def ReloadData():
    index=np.random.choice(range(1,10),2,replace=False)
    
    file_name=r'Feature_File/feature50k_pool3_'+str(index[0])+'_.pkl'
    d=np.load(file_name)
    fea_data=d['feature'].reshape((d['feature'].shape[0],d['feature'].shape[2],d['feature'].shape[3],d['feature'].shape[4],))
    fea_label=d['label']
    file_name=r'Feature_File/feature50k_pool3_'+str(index[1])+'_.pkl'
    d=np.load(file_name)
    f_=d['feature'].reshape((d['feature'].shape[0],d['feature'].shape[2],d['feature'].shape[3],d['feature'].shape[4],))
    fea_data=np.concatenate((fea_data,f_),axis=0)
    fea_label=np.concatenate((fea_label,d['label']),axis=0)
    return fea_data,fea_label

In [ ]:
X=tf.placeholder(tf.float32,[None,8,8,2048])
Y=tf.placeholder(tf.float32,[None,16])
TRAINGING=tf.placeholder(tf.bool)
#
y_socre=model_cnn(X,TRAINGING)
LOSS=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_socre,labels=Y))
TRAIN=tf.train.AdamOptimizer(1e-4).minimize(LOSS)
tf.summary.scalar('loss',LOSS)
#ACC
ACC_C=tf.equal(tf.arg_max(y_socre,1),tf.arg_max(Y,1))
ACC=tf.reduce_mean(tf.cast(ACC_C,tf.float32))
tf.summary.scalar('Accuracy',ACC)
BN_OPS=tf.get_collection(tf.GraphKeys.UPDATE_OPS)
writer_tr=tf.summary.FileWriter('./mylog/CNN/train')
writer_te=tf.summary.FileWriter('./mylog/CNN/test')

In [ ]:
# fea_mat=d['feature'].reshape((-1,2048))
# y_label=d['label']
# fea_mat=feature_data
# y_label=label_data
# index=np.arange(fea_mat.shape[0])
# np.random.shuffle(index)
# tr_index=index[:int(fea_mat.shape[0]*0.8)]
# te_index=index[int(fea_mat.shape[0]*0.8):]
# tr_data=fea_mat[tr_index]
# te_data=fea_mat[te_index]
# tr_label=y_label[tr_index]
# te_label=y_label[te_index]

In [ ]:
merge=tf.summary.merge_all()
init=sess.run(tf.global_variables_initializer())

In [ ]:
d=np.load('Feature_File/fea_validate65k.pkl')
te_data,te_label=d['feature'],d['label']
del d
tr_data,tr_label=ReloadData()

In [ ]:
#1k 大概25min
for i in range(1000*10,1000*10+1000*10*4):
    if i%1000==0:
        print('change traing data')
        del tr_data
        del tr_label
        tr_data,tr_label=ReloadData()
    mask=np.random.choice(range(tr_data.shape[0]),128,replace=False)
    x_,y_=tr_data[mask],tr_label[mask]
    sess.run([TRAIN,BN_OPS],{X:x_,Y:y_,TRAINGING:True })
                  
    if i%10==0:
        loss_,acc_,m_,_=sess.run([LOSS,ACC,merge,BN_OPS],{X:x_,Y:y_,TRAINGING:True})
        writer_tr.add_summary(m_,i)
        print('epoch:{},loss:{},accuracy:{}'.format(i,loss_,acc_))
    if i%20==0:
        mask=np.random.choice(range(te_data.shape[0]),256,replace=False)
        x_,y_=te_data[mask],te_label[mask]
        loss_,acc_,m_,_=sess.run([LOSS,ACC,merge,BN_OPS],{X:x_,Y:y_,TRAINGING:True})
        writer_te.add_summary(m_,i)
        print('--epoch:{},loss:{},accuracy:{}'.format(i,loss_,acc_))
                   

In [ ]:
acc_sum=0
for j_ in range(10):
    mask=np.random.choice(range(te_data.shape[0]),300,replace=False)
    x_,y_=te_data[mask],te_label[mask]
    loss_,acc_,m_,_=sess.run([LOSS,ACC,merge,BN_OPS],{X:x_,Y:y_,TRAINGING:False})
    acc_sum+=acc_

In [ ]:
acc_sum/10

In [ ]:
#保存
saver=tf.train.Saver()
saver.save(sess,'./model_save/inception_pool3_cnn/modelacc_94``'+'.ckpt',global_step=i)

In [ ]:
#复原
#saver.restore(sess,'./model_save/model_mit_cal_1017/model_conv310192042.ckpt-23708')
saver=tf.train.Saver()
saver.restore(sess,'./model_save/inception_pool3_cnn/modelacc_90.ckpt-9999')

In [ ]:
/***************************************************
测试

In [ ]:
import cv2
import time

In [ ]:
def drawline(img_,line_w=1,line_color=(0,0,0),wandh_num=4):
        '''
        在图片上格子
        :param line_w: 线宽
        :param line_color: 线颜色
        :param wandh_num:  长宽线的数量
        :return:  无
        '''
        h,w=img_.shape[0],img_.shape[1]
        w_num,h_num=wandh_num,wandh_num
        h_,w_=h//h_num,w//w_num

        # 竖线 (w,h)
        for i in range(1,w_num):
            #print(i)
            cv2.line(img_,(w_*i,0),(w_*i,h),line_color,line_w)
        # 横线
        for i in range(1,h_num):
            cv2.line(img_,(0,h_*i),(w,h_*i),line_color,line_w)
        return img_

def drew_face_eye2(img):
        '''
        检测用户当前环境是否能够比较好的识别面部特征
        辨识出脸部以及眼部，标记出来
        :param img:
        :return:
        '''
        face_cascade = cv2.CascadeClassifier(r'D:\Proj_DL\Code\Proj_EyeTraker\haarcascade_frontalface_default.xml')
        eye_cascade = cv2.CascadeClassifier(r'D:\Proj_DL\Code\Proj_EyeTraker\haarcascade_eye.xml')
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3,5)
        #确保画面中只有一个人脸识别出来
        if(len(faces)!=1):
            return None
        for (x,y,w,h) in faces:
            cv2.rectangle(img,(x-10,y-10),(x+w+10,y+h+10),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
            #保存脸部图像
            face_mat=roi_color.copy()
            #检测视频中脸部的眼睛，并用vector保存眼睛的坐标、大小（用矩形表示）
            eyes = eye_cascade.detectMultiScale(roi_gray,scaleFactor=1.1, minNeighbors=5, minSize=(27, 27),
                                                flags=cv2.CASCADE_SCALE_IMAGE)
            if len(eyes)!= 2:
                return None
            eye_mat=[]
            for e in eyes:
                xe,ye,we,he=e
                eye_mat.append(face_mat[ye:ye+he,xe:xe+we])
                cv2.rectangle(roi_color,(xe-3,ye-3),(xe+we+3,ye+he+3),(0,0,255),2)
        return  (img,face_mat,eye_mat)
def drawblock(img,line_num,block_id=0,blockcolor=(46,218,255),blockwideth=5,show_rec=True):
        '''
        选定九宫格，在这个格子上填充矩形表示选定这个格子
        :param img_: 图片
        :param block: 九宫格序号 0-15
        :param blockcolor: 矩形框颜色
        :param blockwideth: 框的宽度
        :return:
        '''
        h,w=img.shape[0],img.shape[1]
        w_line,h_line=line_num,line_num
        h_,w_=h//h_line,w//w_line
        cor_h=block_id//line_num
        cor_w=block_id%line_num
        sx,sy=cor_w*w_,cor_h*h_
        
        if show_rec:
        #将整个矩形填充为其他颜色
            img[sy:sy+h_,sx:sx+w_,:]=blockcolor
        else:
            roi_=img[sy:sy+h_,sx:sx+w_]
            cv2.circle(roi_,(roi_.shape[1]//2,roi_.shape[0]//2), 10, (255,128,120), -1)
       
        return img

In [ ]:
tar_img=cv2.imread('test.jpg')
tar_img=drawline(tar_img,wandh_num=4)
tar_img=drawblock(tar_img,line_num=4,block_id=6)
cap=cv2.VideoCapture(0)
_,frame=cap.read()
cap.release()
v_test=tar_img.copy()
tar_img[:480,1280:]=frame
cv2.namedWindow('Capture001',cv2.WINDOW_NORMAL)
cv2.setWindowProperty('Capture001', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
cv2.imshow('Capture001',tar_img)
cv2.waitKey()
cv2.destroyWindow('Capture001')

In [ ]:
prob=tf.nn.softmax(y_socre)
guess=tf.arg_max(prob,1)

In [ ]:
fps=10

fsize_desk=(1920,1080)
# save video
video_d=cv2.VideoWriter('demo_desk.flv',cv2.VideoWriter_fourcc('F','L','V','1'),fps,fsize_desk)

In [ ]:
cap=cv2.VideoCapture(0)
s_time=time.time()
tar_img=cv2.imread('test.jpg')
#设置window 为全屏
cv2.namedWindow('Capture001',cv2.WINDOW_NORMAL)
cv2.setWindowProperty('Capture001', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
#每16帧输出一个最常见结果
frame_counter=0
ret_counter=0
frame_counter_dict={}
block_16id=0
s_time=time.time()
random_show=np.random.randint(0,16,1)[0]
while True:
    ret,fram=cap.read()
    if ret:
        ret_counter+=1
        #显示 内窥镜图像
        tar_img=cv2.imread('test.jpg')
        tar_img=drawline(tar_img,wandh_num=4)
        if ret_counter%18==0:
            random_show=np.random.randint(0,16,1)[0]
        tar_img=drawblock(tar_img,line_num=4,block_id=random_show)
        cv2.imshow('Capture001',tar_img)
       
        #预测注视方向
        face_op=drew_face_eye(fram)
        if face_op == None:
            continue
        _,f,_=face_op
        f=f.reshape((1,f.shape[0],f.shape[1],f.shape[2]))
        fea_=sess.run(feature_bottleneck,{jpeg_tensor:f})
       
        guess_=sess.run(guess,feed_dict={X:fea_,TRAINGING:False})
        if frame_counter==1:
            frame_counter=0
            max_count=0
            for k in frame_counter_dict:
                if frame_counter_dict[k]>max_count:
                    block_16id=k
                    max_count=frame_counter_dict[k]
            frame_counter_dict={}
        else:
            frame_counter+=1
            frame_counter_dict[guess_[0]]=frame_counter_dict.get(guess_[0],0)+1
        
        tar_img=drawblock(tar_img,line_num=4,block_id=block_16id,show_rec=False)
        v_img=tar_img.copy()
        v_img[:480,1280:]=fram
        cv2.imshow('Capture001',tar_img)
        #print(time.time()-s_time)
        #s_time=time.time()
        video_d.write(v_img)
        if cv2.waitKey(30)&0xff==27:
            print('out')
            break
    if (time.time()-s_time)>60*5:
        print('time out')
        break
video_d.release()
cap.release()
cv2.destroyAllWindows()